<a href="https://colab.research.google.com/github/Taeyoung90/pytorch_firststep/blob/main/pytorch_first2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 합성공 신경망

## 모델구현, 학습, 결과 확인

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

DataLoader : 효율적인 학습에 따라 데이터를 어떤 규칙에 따라 정렬하거나 섞거나 해주는 역할  
torchvision : 유명한 영상처리용 데이터셋, 모델, 이미지 변환기가 들어있는 패키지

In [4]:
mnist_train = dset.MNIST('./', train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST('./',train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


transforms.ToTensor() : 이미지를 파이토치 텐서로 변환  
DataLoader의 num_workers : 데이터를 묶을 때 사용할 프로세스 개수
drop_last : 데이터를 batch_size만큼 묶고 나머지를 버릴지

In [2]:
class CNN(nn.Module):
  def __init__(self): #nn.Module 초기화 역할
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1,16,5), #[batch_size,1,28,28] > [batch_size,16,24,24]
        nn.ReLU(),
        nn.Conv2d(16,32,5),#[batch_size,16,24,24] > [batch_size,32,20,20]
        nn.ReLU(),
        nn.MaxPool2d(2,2),#[batch_size,32,20,20] > [batch_size,32,10,10]
        nn.Conv2d(32,64,5),#[batch_size,32,10,10] > [batch_size,64,6,6]
        nn.ReLU(),
        nn.MaxPool2d(2,2)#[batch_size,64,6,6] > [batch_size,64,3,3]
    )
    self.fc_layer =nn.Sequential(
        nn.Linear(64*3*3,100),#[64*3*3] >[100]
        nn.ReLU(),
        nn.Linear(100,10)# [100] > [10]
    )
  
  def forward(self, x): #순차적으로 실행
    out = self.layer(x) #layer에 x를 넣고 나온 출력 값 = out [10]
    out = out.view(batch_size,-1) #[batch_size, -1]형태를 전달, -1은 알아서 계산 > batch_size뒤의 3개 인수 64,3,3을 곱해서 하나로 만듦
    out = self.fc_layer(out)
    return out

mnist data의 입력값 = [batch_size, 1, 28, 28]  
conv(1,16,5) 통과후  
[batch_size, 16, 24,24]  
[4, 16] > tensor.view(2,-1) > [2,32]  
뒤의 -1은 알아서 계산하는 의미

In [3]:
device= torch.device('cude:0' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
loss_arr=[]
for i in range(num_epoch):
  for j, [image,label] in enumerate(train_loader):
    x = image.to(device)# 이미지 데이터 device에 얹고
    y_ = label.to(device)# 라벨도 device에 얹고

    optimizer.zero_grad()#초기화
    output = model.forward(x) #순차적 실행
    loss = loss_func(output,y_)
    loss.backward() #기울기 계산, 역전파
    optimizer.step() #업데이트

    if j %1000 ==0:
      print(loss)
      loss_arr.append(loss.cpu().detach().numpy())

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor(2.3032, grad_fn=<NllLossBackward>)
tensor(0.1928, grad_fn=<NllLossBackward>)
tensor(0.0940, grad_fn=<NllLossBackward>)
tensor(0.1197, grad_fn=<NllLossBackward>)
tensor(0.0611, grad_fn=<NllLossBackward>)
tensor(0.0319, grad_fn=<NllLossBackward>)
tensor(0.0337, grad_fn=<NllLossBackward>)
tensor(0.0650, grad_fn=<NllLossBackward>)
tensor(0.0469, grad_fn=<NllLossBackward>)
tensor(0.0517, grad_fn=<NllLossBackward>)


모델 테스트 데이터에 대해 검증

In [6]:
correct = 0 #맞은 개수 초기화
total = 0 #전체 개수 초기화

with torch.no_grad(): #테스트 모드이기 때문에 기울기 계산 안함
  for image, label in test_loader:
    x= image.to(device)
    y_ = label.to(device)

    output = model.forward(x)
    _, output_index = torch.max(output,1) #최대값(예측값이 가장 큰 값)과 인덱스(해당라벨) 반환

    total += label.size(0) #label의 0번째는 batch_size
    correct += (output_index == y_).sum().float() #output_index와 y_(라벨)같은면 1아니면 0인데 batch_size에 대해서 모두 1,0변환하고 sum하면 결국 맞는것 개수와 동일

  print('accuracy of test data: {}'.format(100*correct/total))

accuracy of test data: 98.74799346923828


In [10]:
label.size()

torch.Size([256])

## VGGNET


- 단순하게 짠 코드  
conv 2번 반복 2개, 3번 반복 3개

In [11]:
def conv_2_block(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=3),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim, kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

def conv_3_block(in_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2,2)
  )
  return model

위 함수를 이용해 16개 레이어를 가진 전체 모델 생성  
이 구현은 블록 단위로 순서대로 읽기에는 편하지만, 11개나 19개 레이어를 생성하는 경우 큰 수정을 해야함

In [12]:
class VGG(nn.Module):
  def __init__(self, base_dim, num_classes=2):
    super(VGG, self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3, base_dim),
        conv_2_block(base_dim, 2*base_dim),
        conv_3_block(2*base_dim, 4*base_dim),
        conv_3_block(4*base_dim, 8*base_dim),
        conv_3_block(8*base_dim, 8*base_dim),
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(8*base_dim*7*7, 100),
        nn.ReLU(True),
        nn.Linear(100,20),
        nn.ReLU(True),
        nn.Linear(20, num_classes),
    )

  def forward(self, x):
    x =self.feature(x)
    x = x.view(x.size(0), -1)
    x= self.fc_layer(x)
    return x

- 파이토치 공식 구현 버전  
(코드는 https://github.com/pytorch/vision/tree/master/torchvision/models에서 찾을 수 있음

In [17]:
class VGG(nn.Module):

  def __init__(self, features, num_classes=1000, init_weights=True):
    super(VGG,self).__init__()
    self.features= features
    self.avgpool = nn.AdaptiveAvgPool2d((7,7))
    self.classifier = nn.Sequential(
        nn.Linear(512*7*7, 4096),
        nn.ReLU(True),
        nn.Linear(4096, 4096),
        nn.ReLU(True),
        nn.Linear(4096, num_classes),
    )
    if init_weights:
      self._initialize_weights()

  def forward(self, x):
    x= self.features(x)
    x= self.avgpool(x)
    x = x.view(x.size(0), -1)
    x =self.classifier(x)

  def _initialize_weights(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.constant_(m.bias, 0)

  def make_layers(cfg, batch_norm=False):
    layers= [] # 빈리스트 생성
    in_channels=3
    for v in cfg:
      if v == 'M': # cfg요소가 M 이면 maxpooling 추가
        layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
      else:
        conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
        if batch_norm:
          layers+= [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
        else:
          layers += [conv2d, nn.ReLU(inplace=True)]
        in_channels = v
    return nn.Sequential(*layers) # *는 layers리스트의 요소를 모두 꺼냄을 의미

  cfgs = {
      'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
      'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
      'D': [64, 64, 'M', 128, 128, 'M', 256, 256,256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
      'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256,256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'] 
  }

단순버젼과 공식 구현의 가장 큰 차이 : VGG 클래스의 features에 들어가는 모델을 make_layers라는 함수로 생성  
모델 구조를 cfgs에 미리 정의(숫자는 필터수, M은 맥스풀링 의미)  
make_layers함수 : 모델 구조를 인수로 받아서 먼저 

In [13]:
a= [1,2,3]

In [15]:
b=[*a]
b

[1, 2, 3]